## Chapter 13 자료 연계 (Record Linkage) 실습

## 사용 데이터: Naver, local 데이터

In [1]:
import pandas as pd
local = pd.read_csv('Local_Restaurant.csv', encoding='utf-8')
Naver = pd.read_csv('Naver_Restaurant.csv', encoding='utf-8')
local.head()

,상호명,주소,우편번호,인허가일자,폐업일자,규모
0,오향만두,서울특별시 서대문구 연희맛로 22 (연희동),3708,20200602,NaN,59.92
1,돼지주막,"서울특별시 서대문구 거북골로 25-6 (남가좌동,지상1층)",3665,20021216,NaN,89.02
2,수락간,"서울특별시 서대문구 홍제천로6길 4 (연희동, 1층)",3700,20170919,NaN,49.50
3,깡통집,서울특별시 서대문구 증가로10길 62 (홍은동),3662,20140812,NaN,34.00
4,비어포차,"서울특별시 서대문구 홍은중앙로8길 24, 1층 (홍은동)",3605,20190724,NaN,69.42


In [2]:
Naver.head()

,ID,상호명,포장,예약,주소,평균가격,최소가격,최대가격
0,11560770,압구정샤브샤브,0,0,서울 서대문구 연대동문길 26,14500.0,13000.0,16000.0
1,11617019,아이비사주카페,0,0,서울 서대문구 이화여대7길 23,9600.0,5000.0,20000.0
2,11625126,피자헛 홍제본점,1,0,서울 서대문구 세검정로 73,23804.0,19900.0,25130.0
3,11710118,대가감자탕&보쌈,0,0,서울 서대문구 홍제내길 136,19500.0,14000.0,25000.0
4,11646388,아마이,0,0,서울 서대문구 연세로 21,24000.0,16000.0,32000.0


## 1. 전처리

In [3]:
Naver['구이하주소'] = Naver['주소'].map(lambda x: ' '.join(x.split(' ')[1:]))
local['구이하주소'] = local['주소'].map(lambda x: ' '.join(x.split(' ')[1:]))
local['구이하주소'] = local['구이하주소'].map(lambda x: x[:x.find('(')].strip() )
local.head()

,상호명,주소,우편번호,인허가일자,폐업일자,규모,구이하주소
0,오향만두,서울특별시 서대문구 연희맛로 22 (연희동),3708,20200602,NaN,59.92,서대문구 연희맛로 22
1,돼지주막,"서울특별시 서대문구 거북골로 25-6 (남가좌동,지상1층)",3665,20021216,NaN,89.02,서대문구 거북골로 25-6
2,수락간,"서울특별시 서대문구 홍제천로6길 4 (연희동, 1층)",3700,20170919,NaN,49.50,서대문구 홍제천로6길 4
3,깡통집,서울특별시 서대문구 증가로10길 62 (홍은동),3662,20140812,NaN,34.00,서대문구 증가로10길 62
4,비어포차,"서울특별시 서대문구 홍은중앙로8길 24, 1층 (홍은동)",3605,20190724,NaN,69.42,"서대문구 홍은중앙로8길 24, 1층"


In [4]:
Naver.head()

,ID,상호명,포장,예약,주소,평균가격,최소가격,최대가격,구이하주소
0,11560770,압구정샤브샤브,0,0,서울 서대문구 연대동문길 26,14500.0,13000.0,16000.0,서대문구 연대동문길 26
1,11617019,아이비사주카페,0,0,서울 서대문구 이화여대7길 23,9600.0,5000.0,20000.0,서대문구 이화여대7길 23
2,11625126,피자헛 홍제본점,1,0,서울 서대문구 세검정로 73,23804.0,19900.0,25130.0,서대문구 세검정로 73
3,11710118,대가감자탕&보쌈,0,0,서울 서대문구 홍제내길 136,19500.0,14000.0,25000.0,서대문구 홍제내길 136
4,11646388,아마이,0,0,서울 서대문구 연세로 21,24000.0,16000.0,32000.0,서대문구 연세로 21


In [5]:
Naver.index = [f'rec-Naver-{i}' for i in range(Naver.shape[0])]
local.index = [f'rec-local-{i}' for i in range(local.shape[0])]

Naver.index.name = 'rec_id'
local.index.name = 'rec_id'
local.head()

,상호명,주소,우편번호,인허가일자,폐업일자,규모,구이하주소
rec_id,,,,,,,
rec-local-0,오향만두,서울특별시 서대문구 연희맛로 22 (연희동),3708,20200602,NaN,59.92,서대문구 연희맛로 22
rec-local-1,돼지주막,"서울특별시 서대문구 거북골로 25-6 (남가좌동,지상1층)",3665,20021216,NaN,89.02,서대문구 거북골로 25-6
rec-local-2,수락간,"서울특별시 서대문구 홍제천로6길 4 (연희동, 1층)",3700,20170919,NaN,49.50,서대문구 홍제천로6길 4
rec-local-3,깡통집,서울특별시 서대문구 증가로10길 62 (홍은동),3662,20140812,NaN,34.00,서대문구 증가로10길 62
rec-local-4,비어포차,"서울특별시 서대문구 홍은중앙로8길 24, 1층 (홍은동)",3605,20190724,NaN,69.42,"서대문구 홍은중앙로8길 24, 1층"


In [6]:
Naver.head()

,ID,상호명,포장,예약,주소,평균가격,최소가격,최대가격,구이하주소
rec_id,,,,,,,,,
rec-Naver-0,11560770,압구정샤브샤브,0,0,서울 서대문구 연대동문길 26,14500.0,13000.0,16000.0,서대문구 연대동문길 26
rec-Naver-1,11617019,아이비사주카페,0,0,서울 서대문구 이화여대7길 23,9600.0,5000.0,20000.0,서대문구 이화여대7길 23
rec-Naver-2,11625126,피자헛 홍제본점,1,0,서울 서대문구 세검정로 73,23804.0,19900.0,25130.0,서대문구 세검정로 73
rec-Naver-3,11710118,대가감자탕&보쌈,0,0,서울 서대문구 홍제내길 136,19500.0,14000.0,25000.0,서대문구 홍제내길 136
rec-Naver-4,11646388,아마이,0,0,서울 서대문구 연세로 21,24000.0,16000.0,32000.0,서대문구 연세로 21


## 2. blocking

In [7]:
Naver['구'] = Naver['구이하주소'].map(lambda x: x.split(' ')[0])
local['구'] = local['구이하주소'].map(lambda x: x.split(' ')[0])

Naver['구미만주소'] = Naver['구이하주소'].map(lambda x: ' '.join(x.split(' ')[1:]) )
local['구미만주소'] = local['구이하주소'].map(lambda x: ' '.join(x.split(' ')[1:]) )
local.head()

,상호명,주소,우편번호,인허가일자,폐업일자,규모,구이하주소,구,구미만주소
rec_id,,,,,,,,,
rec-local-0,오향만두,서울특별시 서대문구 연희맛로 22 (연희동),3708,20200602,NaN,59.92,서대문구 연희맛로 22,서대문구,연희맛로 22
rec-local-1,돼지주막,"서울특별시 서대문구 거북골로 25-6 (남가좌동,지상1층)",3665,20021216,NaN,89.02,서대문구 거북골로 25-6,서대문구,거북골로 25-6
rec-local-2,수락간,"서울특별시 서대문구 홍제천로6길 4 (연희동, 1층)",3700,20170919,NaN,49.50,서대문구 홍제천로6길 4,서대문구,홍제천로6길 4
rec-local-3,깡통집,서울특별시 서대문구 증가로10길 62 (홍은동),3662,20140812,NaN,34.00,서대문구 증가로10길 62,서대문구,증가로10길 62
rec-local-4,비어포차,"서울특별시 서대문구 홍은중앙로8길 24, 1층 (홍은동)",3605,20190724,NaN,69.42,"서대문구 홍은중앙로8길 24, 1층",서대문구,"홍은중앙로8길 24, 1층"


In [8]:
Naver.head()

,ID,상호명,포장,예약,주소,평균가격,최소가격,최대가격,구이하주소,구,구미만주소
rec_id,,,,,,,,,,,
rec-Naver-0,11560770,압구정샤브샤브,0,0,서울 서대문구 연대동문길 26,14500.0,13000.0,16000.0,서대문구 연대동문길 26,서대문구,연대동문길 26
rec-Naver-1,11617019,아이비사주카페,0,0,서울 서대문구 이화여대7길 23,9600.0,5000.0,20000.0,서대문구 이화여대7길 23,서대문구,이화여대7길 23
rec-Naver-2,11625126,피자헛 홍제본점,1,0,서울 서대문구 세검정로 73,23804.0,19900.0,25130.0,서대문구 세검정로 73,서대문구,세검정로 73
rec-Naver-3,11710118,대가감자탕&보쌈,0,0,서울 서대문구 홍제내길 136,19500.0,14000.0,25000.0,서대문구 홍제내길 136,서대문구,홍제내길 136
rec-Naver-4,11646388,아마이,0,0,서울 서대문구 연세로 21,24000.0,16000.0,32000.0,서대문구 연세로 21,서대문구,연세로 21


In [9]:
import recordlinkage
indexer = recordlinkage.Index()
# Blocking 없이하는 경우
indexer.full()
candidate_links = indexer.index(Naver, local)
len(candidate_links)

543300

In [10]:
indexer = recordlinkage.Index()
# 구 정보로 blocking할 경우
indexer.block('구')
candidate_links = indexer.index(Naver, local)
len(candidate_links)

166200

## 3-1. 변수 만들기: 기준 생성 (comparing)

In [11]:
compare_cl = recordlinkage.Compare()
compare_cl.string('구미만주소', '구미만주소', method='jarowinkler', threshold=0.5, label='구미만주소')
compare_cl.string('상호명', '상호명', method='jarowinkler', threshold=0.85, label='상호명' )

<Compare>

## 3-2. 변수 만들기: 변수 생성

In [12]:
features = compare_cl.compute(candidate_links, Naver, local)
features.head()

구미만주소  상호명
rec_id_1    rec_id_2               
rec-Naver-0 rec-local-0    1.0  0.0
            rec-local-1    1.0  0.0
            rec-local-2    0.0  0.0
            rec-local-3    1.0  0.0
            rec-local-4    1.0  0.0

## 4.  자료 연계 알고리즘 실행 (ECM-Algorithm)

In [13]:
from recordlinkage import ECMClassifier
ecm = ECMClassifier(binarize=0.5)
result_ecm_with_blocking = ecm.fit_predict(features)
len(result_ecm_with_blocking)

39

In [14]:
result_ecm_with_blocking

MultiIndex([(  'rec-Naver-89', 'rec-local-94'),
            ( 'rec-Naver-134', 'rec-local-14'),
            ( 'rec-Naver-140', 'rec-local-66'),
            ( 'rec-Naver-185',  'rec-local-9'),
            ( 'rec-Naver-191', 'rec-local-51'),
            ( 'rec-Naver-373',  'rec-local-3'),
            ( 'rec-Naver-407',  'rec-local-3'),
            ( 'rec-Naver-532', 'rec-local-76'),
            ( 'rec-Naver-753', 'rec-local-53'),
            ( 'rec-Naver-755',  'rec-local-7'),
            ( 'rec-Naver-855', 'rec-local-61'),
            ( 'rec-Naver-861', 'rec-local-50'),
            ( 'rec-Naver-996',  'rec-local-6'),
            ('rec-Naver-1106',  'rec-local-0'),
            ('rec-Naver-1156', 'rec-local-27'),
            ('rec-Naver-1232', 'rec-local-46'),
            ('rec-Naver-1326', 'rec-local-72'),
            ('rec-Naver-1543',  'rec-local-6'),
            ('rec-Naver-1545',  'rec-local-4'),
            ('rec-Naver-1554', 'rec-local-74'),
            ('rec-Naver-2004', 'rec-loca

## 5. 자료연계 결과

In [15]:
local.loc[['rec-local-94']]

,상호명,주소,우편번호,인허가일자,폐업일자,규모,구이하주소,구,구미만주소
rec_id,,,,,,,,,
rec-local-94,이정희떡볶이,"서울특별시 서대문구 증가로10길 14 (남가좌동,(지상1층))",3665,20030312,NaN,18.4,서대문구 증가로10길 14,서대문구,증가로10길 14


In [16]:
Naver.loc[['rec-Naver-89']]

,ID,상호명,포장,예약,주소,평균가격,최소가격,최대가격,구이하주소,구,구미만주소
rec_id,,,,,,,,,,,
rec-Naver-89,11856504,이정희떡볶이,0,0,서울 서대문구 증가로10길 14,2600.0,1000.0,5000.0,서대문구 증가로10길 14,서대문구,증가로10길 14
